### Generates bespoke MLP-like QBAFs

In [61]:
import random
random.seed(1)
import sys
sys.path.append("../../src/")

In [62]:
def generate_random_mlp_graph(layer_sizes, connection_prob):
    """
    generate a random MLP-like QBAF structure, represented by DAG

    parameter:
    - connection_prob: The probability of inter-layer connections (1.0 = fully connected, 0.0 = no connections)

    return:
    - graph: MLP structure in adjacent list (DAG)
    """

    graph = {}  # Store the DAG using an adjacency list
    node_id = 0  # neuron ID
    layer_nodes = []  # Record the neuron IDs in each layer

    # Create neuron nodes
    for size in layer_sizes:
        layer = [node_id + i for i in range(size)]
        layer_nodes.append(layer)
        node_id += size

    # Generate inter-layer connections
    for i in range(len(layer_nodes) - 1):  # Layer-by-layer connection
        for src in layer_nodes[i]:  # Current layer neurons
            for dst in layer_nodes[i+1]:  # Next layer neurons
                if random.uniform(0, 1) < connection_prob:
                    graph.setdefault(src, set()).add(dst)

    # if node not in graph, add empty set
    for layer in layer_nodes:
        for node in layer:
            graph.setdefault(node, set())

    return graph

In [63]:
def get_layer_nodes(layer_sizes, layer_index=None):
    node_id = 0
    for i, size in enumerate(layer_sizes):
        if i == layer_index:
            return [str(n) for n in range(node_id, node_id + size)]
        node_id += size

    raise ValueError(f"Invalid layer_index {layer_index}: must be between 0 and {len(layer_sizes) - 1}")

In [64]:
# generate a random MLP-QBAF and output to a file
def generate_and_write_graph(filename, layer_sizes, connection_prob):
    with open(filename, 'w') as f:
        sys.stdout = f

        # generate the node and edge of a graph
        random_graph = generate_random_mlp_graph(layer_sizes, connection_prob)

        L0 = get_layer_nodes(layer_sizes, len(layer_sizes)-1) # output layer
        L1 = get_layer_nodes(layer_sizes, len(layer_sizes)-2) # last hidden layer
        L2 = get_layer_nodes(layer_sizes, len(layer_sizes)-3)
        L3 = get_layer_nodes(layer_sizes, len(layer_sizes)-4)
        if len(layer_sizes)>=5:
            L4 = get_layer_nodes(layer_sizes, len(layer_sizes)-5)
            L4 = list(map(int, L4))
        L0 = list(map(int, L0))
        L1 = list(map(int, L1))
        L2 = list(map(int, L2))
        L3 = list(map(int, L3))

        # generate base scores for arguments
        for node, edges in random_graph.items():
            # print(node)
            if node in L1:
                random_float = round(random.uniform(0.0, 0.1),2)
            else:
                random_float = round(random.uniform(0.0, 1.0),2)
            print(f"arg({node}, {random_float}).")

        # generate polarity for edges
        for node, edges in random_graph.items():
            for edge in edges:

                if edge in L1:
                    random_boolean = True
                elif edge in L2:
                    random_boolean = False
                else:
                    random_boolean = random.choice([True, False])
                if random_boolean:
                    print(f"att({node}, {edge}).")
                else:
                    print(f"sup({node}, {edge}).")

    sys.stdout = sys.__stdout__

In [65]:
N = 100 # generate N QBAFs storing in N files
connection_prob=1.0
layer_sizes = [8,64,16,8,8]
for i in range(N):
    filename = f'../../bags/mlp_{i}.bag'
    generate_and_write_graph(filename, layer_sizes, connection_prob)